In [1]:
require 'cunn'

----------------------------------------------------------------------

tar = 'http://torch7.s3-website-us-east-1.amazonaws.com/data/mnist.t7.tgz'

if not paths.dirp('mnist.t7') then
   os.execute('wget ' .. tar)
   os.execute('tar xvf ' .. paths.basename(tar))
end

trainFile = 'mnist.t7/train_32x32.t7'
testFile = 'mnist.t7/test_32x32.t7'

----------------------------------------------------------------------

print '==> loading dataset'

trainData = nil
testData = nil
train = torch.load(trainFile,'ascii')
test = torch.load(testFile,'ascii')
----------------------------------------------------------------------

==> loading dataset	


In [2]:
trainData = {data = train.data[{ {1,6000}, {}, {} }],
             labels = train.labels[ { {1, 6000} } ]}
testData = {data = test.data[{ {1,1000}, {}, {} }],
             labels = test.labels[ { {1, 1000} } ]}

In [3]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(trainData, 
    {__index = function(t, i) 
                    return {t.data[i], t.labels[i]} 
                end}
);

function trainData:size() 
    return self.data:size(1)*0.1 
end

In [4]:
trainData.data = trainData.data:cuda():div(255)
mean = trainData.data:mean()
trainData.data = trainData.data:add(-mean)
trainData.labels = trainData.labels:cuda()

testData.data = testData.data:cuda():div(255)
testData.data = testData.data:add(-mean)
testData.labels = testData.labels:cuda()

trsize = trainData.data:size(1)
tesize = testData.data:size(1)

In [5]:
{testData}

{
  1 : 
    {
      data : CudaTensor - size: 1000x1x32x32
      labels : CudaTensor - size: 1000
    }
}


In [6]:
-- cnn structure
require 'nn'

net = nn.Sequential()
net:add(nn.SpatialConvolution(1, 6, 5, 5)) -- 1 input image channel, 6 output channels, 5x5 convolution kernel
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.
net:add(nn.SpatialConvolution(6, 16, 5, 5))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.View(16*5*5))                    -- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
net:add(nn.Linear(16*5*5, 120))             -- fully connected layer (matrix multiplication between input and weights)
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(84, 10))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
net:add(nn.LogSoftMax())                     -- converts the output to a log-probability. Useful for classification problems
net = net:cuda()
-- print('Lenet5\n' .. net:__tostring());

In [7]:
criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class
criterion = criterion:cuda()

In [8]:
trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 200 -- just do 5 epochs of training.
trainer:train(trainData)

# StochasticGradient: training	


# current error = 2.2990066736937	


# current error = 2.1771078719695	


# current error = 1.3374640496572	


# current error = 0.65499614377817	


...

# current error = 1.7526944478353e-05	


# current error = 1.7414093017578e-05	


# current error = 1.7321904500326e-05	


# current error = 1.7201105753581e-05	


# current error = 1.7086664835612e-05	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 1.7086664835612e-05	


In [9]:
correct = 0
for i=1,1000 do
    local groundtruth = testData.labels[i]
    local prediction = net:forward(testData.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    if groundtruth == indices[1] then
        correct = correct + 1
    end
end

print(correct, 100*correct/1000 .. ' % ')

910	91 % 	


In [15]:
class_performance = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0}
for i=1,1000 do
    local groundtruth = testData.labels[i]
    local prediction = net:forward(testData.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    if groundtruth == indices[1] then
        class_performance[groundtruth] = class_performance[groundtruth] + 1
    end
end

In [17]:
classes = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
for i=1,#classes do
    print(classes[i], 100*class_performance[i]/100 .. ' %')
end

0	82 %	
1	124 %	
2	104 %	
3	94 %	
4	98 %	
5	80 %	
6	80 %	
7	86 %	
8	76 %	
9	86 %	
